#### Import the required libraries

In [ ]:
import psycopg2 as psy
import pandas as pd
import json
import os
import time

#### Establish connection with the PostgreSQL database

In [ ]:
database_host = "localhost"
database_name = "dbl_data_challenge"
database_user = "admin"
database_pass = "vZtbqKNXGz27cQCH"

In [ ]:
con = psy.connect(
    host = database_host,
    database = database_name,
    user = database_user,
    password = database_pass
)
cur = con.cursor()

#### This cell gets all the tweet ids and the tweet ids of the tweets they're replying to, creates a pair dictionary, which is subsequently used to retrieve the start of a conversation. The conversation id is the id of the tweet which forms the root of the conversation (the starting tweet). When iterating over the pairs, the tweets are added to the converstaion id key in the dictionary which they belong to

In [ ]:
cur.execute("""
    SELECT id_str, in_reply_to_status_id_str FROM tweets
""")

data = cur.fetchall()

convo_pairs = {}
counter = 0

for row in data: 
    convo_pairs[row[0]] = row[1]

total_count = len(convo_pairs)

conversations = []

start = time.time()

for twt_id in convo_pairs: 
    counter += 1
    found_last = False
    convo_id = None
    current = twt_id
    current_fallback = None
    is_snippet = False
    while not found_last: 
        try: 
            if convo_pairs[current] != None: 
                current_fallback = current
                current = convo_pairs[current]
            else: 
                found_last = True
        except KeyError: 
            convo_pairs[current_fallback] = None
            convo_id = current_fallback
            is_snippet = True
            found_last = True
    convo_id = current
    conversations.append((twt_id, convo_id, is_snippet))
    if counter % 100000 == 0: 
        print("[Pairing] Currently at tweet: "+twt_id+" ("+str(counter)+"/"+str(total_count)+") ("+str(round((time.time()-start), 2))+"s)")

counter = 0

start = time.time()

cur.execute("""
    DROP TABLE IF EXISTS conversations
""")

cur.execute("""
    CREATE TABLE conversations (tweet_id VARCHAR(50), conversation_id VARCHAR(50), is_snippet BOOLEAN)
""")

con.commit()

for twt_tuple in conversations: 
    counter += 1
    cur.execute("""
        INSERT INTO conversations (tweet_id, conversation_id, is_snippet) VALUES (%s, %s, %s)
    """, (twt_tuple[0], twt_tuple[1], twt_tuple[2]))
    if counter % 100000 == 0: 
        print("[SQL Insert] Currently at tweet: "+twt_tuple[0]+" ("+str(counter)+"/"+str(total_count)+") ("+str(round((time.time()-start), 2))+"s)")
        con.commit()

print("[SQL Insert] Completed update of data in "+str(round((time.time()-start), 2))+"s")

con.commit()

#### A function which is a able to return all the tweets from a certain conversation id, presented in a dataframe

In [ ]:
def GetConversation(convo_id): 
    columns = ["created_at","timestamp_ms","userid_str","id_str","full_text","in_reply_to_status_id_str","in_reply_to_user_id_str","quoted_status_id_str","quote_count","reply_count","retweet_count","favorite_count","hashtags","user_mentions","conversation_id","thread_id"]
    cur.execute("""
        SELECT tweet.* FROM tweet, conversations
        WHERE tweet.id_str = conversations.tweet_id AND conversations.conversation_id = %(convo_id)s
    """, {"convo_id": convo_id})
    df_tweets = pd.DataFrame(data=cur.fetchall(), columns=columns)
    return df_tweets

GetConversation("1149272217349427203")